# Testing of Model with InferenceSlicer
The following code is the application of the final model on an equal test data set for the comparison of full-frame inference to tile-inference.
The achieved results are equal to the ones achieved with this code.

In [29]:
import os
import cv2
import numpy as np
from pathlib import Path
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import supervision as sv
from ultralytics import YOLO
from collections import defaultdict

# === Konfiguration ===
IMAGE_DIR = Path("test/images")
LABEL_DIR = Path("test/labels")
MODEL_PATH = "Yolo12s_finetuned.pt"
IOU_THRESHOLD = 0.4
CONF_THRESHOLD = 0.45
CLASS_NAMES = {0: "ball", 1: "player", 2: "referee"}

# === Modell laden ===
model = YOLO(MODEL_PATH)

def yolo_to_xyxy(yolo_box, img_w, img_h):
    cls_id, x_center, y_center, w, h = map(float, yolo_box)
    x1 = (x_center - w / 2) * img_w
    y1 = (y_center - h / 2) * img_h
    x2 = (x_center + w / 2) * img_w
    y2 = (y_center + h / 2) * img_h
    return [int(cls_id), x1, y1, x2, y2]

def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    if interArea == 0:
        return 0.0
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def run_tilesliced_inference(image):
    def callback(patch):
        result = model.predict(source=patch, conf=CONF_THRESHOLD, verbose=False)[0]
        return sv.Detections.from_ultralytics(result)

    height, width, _ = image.shape
    tile_w, tile_h = width // 3, height // 2

    slicer = sv.InferenceSlicer(
        callback=callback,
        slice_wh=(tile_w, tile_h),
        overlap_wh=(int(tile_w * 0.3), int(tile_h * 0.3)),
        overlap_ratio_wh=None,
        overlap_filter=sv.OverlapFilter.NON_MAX_SUPPRESSION,
        iou_threshold=0.25,
    )
    return slicer(image)

# === Evaluation vorbereiten ===
all_gt = []
all_pred = []

image_files = sorted(IMAGE_DIR.glob("*.jpg"))

for image_path in image_files:
    image = cv2.imread(str(image_path))
    if image is None:
        continue

    h, w, _ = image.shape
    detections = run_tilesliced_inference(image)

    filtered_boxes = []
    max_ball_conf = -1
    max_ball_box = None

    for box, conf, cls in zip(detections.xyxy, detections.confidence, detections.class_id):
        if cls == 0:  # Ball
            if conf > max_ball_conf:
                max_ball_conf = conf
                max_ball_box = (box, conf, cls)
        elif cls in [1, 2]:  # Player or Referee
            filtered_boxes.append((box, conf, cls))

    if max_ball_box:
        filtered_boxes.append(max_ball_box)

    label_path = LABEL_DIR / (image_path.stem + ".txt")
    gt_boxes = []
    if label_path.exists():
        with open(label_path, "r") as f:
            for line in f:
                gt_boxes.append(yolo_to_xyxy(line.strip().split(), w, h))

    matched_gt = set()
    matched_pred = set()

    for pred_idx, (pred_box, _, pred_cls) in enumerate(filtered_boxes):
        pred_box = pred_box.astype(float)
        for gt_idx, gt in enumerate(gt_boxes):
            gt_cls, *gt_box = gt
            if gt_cls != pred_cls:
                continue
            iou = compute_iou(pred_box, gt_box)
            if iou >= IOU_THRESHOLD and gt_idx not in matched_gt:
                matched_gt.add(gt_idx)
                matched_pred.add(pred_idx)
                all_gt.append(gt_cls)
                all_pred.append(pred_cls)
                break
        else:
            all_pred.append(pred_cls)
            all_gt.append(-1)  # False Positive

    for gt_idx, gt in enumerate(gt_boxes):
        if gt_idx not in matched_gt:
            gt_cls = gt[0]
            all_gt.append(gt_cls)
            all_pred.append(-1)

# === Metriken berechnen ===
unique_classes = sorted(CLASS_NAMES.keys())
results = []
for cls in unique_classes:
    y_true = [1 if g == cls else 0 for g in all_gt]
    y_pred = [1 if p == cls else 0 for p in all_pred]
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    results.append({
        "class": CLASS_NAMES[cls],
        "precision": round(precision, 3),
        "recall": round(recall, 3),
        "f1": round(f1, 3)
    })

print(results)

[{'class': 'ball', 'precision': 0.731, 'recall': 0.633, 'f1': 0.679}, {'class': 'player', 'precision': 0.962, 'recall': 0.985, 'f1': 0.973}, {'class': 'referee', 'precision': 0.889, 'recall': 0.963, 'f1': 0.925}]
